In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [327]:
from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.model_selection import  train_test_split
import time #helper libraries
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from numpy import newaxis
import math
import talib

ModuleNotFoundError: No module named 'talib'

In [325]:
fundamentals = pd.read_csv('../input/fundamentals.csv')
stock_fundamentals = fundamentals.loc[fundamentals['Ticker Symbol'] == 'AAPL'].copy()
stock_fundamentals.resample('D')
stock_fundamentals.head()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [326]:
prices = pd.read_csv('../input/prices-split-adjusted.csv') #get data
stock = prices.loc[prices['symbol'] == 'AAPL'].copy() #get only GOOG prices
stock["y"] = (stock.close - stock.close.shift(1))/stock.close.shift(1) #create returns (called y since that's what we're trying to predict)

#Add features
for i in range(1, LOOK_BACK+1):
    #Add column for returns of n days ago (e.g. yesterday's returns, the returns of the day before yesterday...)
    stock[str(i) + " Days Ago"] = stock["y"].shift(i)
stock = stock[['y'] + [str(i) + " Days Ago" for i in range(1, LOOK_BACK + 1)]].copy().dropna() #drop the other columns

#Scale and separate test and train data
TEST_SIZE = 0.33
train_stop = math.floor((1-TEST_SIZE)*len(stock.index)) #index of where test data starts and training data stops
train = stock.loc[stock.index[0:train_stop]]
for col in stock.columns: 
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(train[[col]]) #!!!!!!!!!!!!!!!! ONLY FIT SCALER ON TRAINING DATA, FITTING ON TEST DATA AS WELL IS CHEATING !!!!!!!!!!!!
    stock[col] = scaler.transform(stock[[col]]) #apply scaler to all data 

#Actually separate test data
stock = np.array(stock.values)
x_train = stock[:train_stop, 1:]
x_test = stock[train_stop:, 1:]
y_train = stock[:train_stop, 0]
y_test = stock[train_stop:, 0]
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

#Define model architecture (copied from someone else's model, will optimize with a genetic algorithm later)
model = Sequential()
model.add(LSTM(input_shape=(1, x_train.shape[2]),units = 50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.add(Activation('linear'))

#Compile, fit on training data, and evaluate on test data
model.compile(loss='mse', optimizer='rmsprop')
model.fit(x_train, y_train, batch_size = 128, epochs=10, validation_split = 0.05, verbose = 0)
model.evaluate(x_test, y_test, verbose = 0)
y_predict = model.predict(x_test)
returns = []
for i in range(len(x_test) - 1):
    if y_predict[i][0] > x_test[i][0][-1]:
        returns.append((y_test[i]-x_test[i][0][-1])/x_test[i][0][-1])
    else:
        returns.append(0)
sharpe = np.mean(returns)/np.std(returns)
print("Sharpe Ratio: ", sharpe)

            date symbol       open    ...          high       volume         y
254   2010-01-04   AAPL  30.490000    ...     30.642857  123432400.0       NaN
721   2010-01-05   AAPL  30.657143    ...     30.798571  150476200.0  0.001729
1189  2010-01-06   AAPL  30.625713    ...     30.747143  138040000.0 -0.015906
1657  2010-01-07   AAPL  30.250000    ...     30.285715  119282800.0 -0.001849
2125  2010-01-08   AAPL  30.042856    ...     30.285715  111902700.0  0.006648

[5 rows x 8 columns]
              date symbol    ...         volume         y
848767  2016-12-23   AAPL    ...     14249500.0  0.001978
849267  2016-12-27   AAPL    ...     18296900.0  0.006351
849767  2016-12-28   AAPL    ...     20905900.0 -0.004264
850267  2016-12-29   AAPL    ...     15039500.0 -0.000257
850767  2016-12-30   AAPL    ...     30586300.0 -0.007796

[5 rows x 8 columns]
Sharpe Ratio:  0.3493611507277231
